In [33]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [34]:
from __future__ import division

import gym
import numpy as np
import pandas as pd
import random
import tensorflow as tf  

%load_ext tensorboard


import matplotlib.pyplot as plt
%matplotlib inline


import tensorflow_addons.layers as layers


from gym import spaces

try:
    import cPickle as pickle
except:
    import pickle


import math

try:
    xrange = xrange
except:
    xrange = range

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [35]:
df_pca = pd.read_csv('./osi_pca_final.csv')

In [36]:
df_pca.head(10)

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,PC11,PC12,PC13,PC14,PC15,PC16,Revenue
0,0.104711,0.115576,-0.395859,-0.367811,0.792326,0.687524,-0.413624,0.404863,-0.295999,-0.032112,-0.020025,0.141937,0.128971,-0.100672,-0.406829,0.845749,0
1,0.000601,0.142662,-0.271433,-0.129434,0.178537,-0.074198,-0.258909,0.315821,-0.120330,0.054817,0.010389,0.211168,0.171537,0.018455,-0.577434,0.774437,0
2,0.110375,0.146153,-0.392492,-0.241203,0.771216,0.936403,0.567604,0.373728,-0.208422,-0.012134,-0.036700,0.155926,0.128702,0.008547,-0.345934,0.866735,0
3,0.034761,0.133210,-0.310407,-0.177025,0.365475,0.202037,-0.171265,0.328262,-0.037567,0.074858,-0.003444,0.192169,0.157149,0.051173,-0.515762,0.798940,0
4,-0.123149,0.048001,0.711157,-0.095758,0.307547,-0.108562,-0.230177,0.309132,0.017062,0.116845,0.001683,0.205523,0.176376,0.135875,-0.541553,0.803390,0
5,-0.023538,0.142550,-0.244491,-0.076902,0.053693,-0.219202,-0.229715,0.302661,-0.056145,0.077405,0.014214,0.208672,0.174958,-0.000499,-0.546529,0.798287,0
6,0.183102,0.084289,-0.393380,-0.326629,0.790718,0.774777,-0.155713,0.354513,-0.149013,0.013143,-0.041774,0.523529,0.119604,0.115097,-0.333076,0.805666,0
7,-0.005150,0.012790,0.571897,-0.358637,0.986430,0.731866,-0.398297,0.403194,-0.096423,0.043948,-0.030007,0.131088,0.136232,-0.075089,-0.399655,0.872121,0
8,0.151072,0.067793,-0.260093,-0.120412,0.182449,-0.026001,-0.147373,0.247943,-0.057471,0.051713,-0.017834,0.966725,0.171442,-0.022431,-0.433824,0.671264,0
9,0.045417,0.112371,-0.232892,-0.063361,0.016315,-0.258814,-0.220094,0.264003,-0.062110,0.074123,0.001389,0.595660,0.171618,0.138461,-0.502776,0.737497,0


In [37]:
# Dataset dummied and scaled in other notebook
df = pd.read_csv('./osi_scaled_clean.csv')

In [38]:
df.Revenue = df.Revenue.apply(lambda x: 1 if x else 0)

In [39]:
df.head(10)

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,...,Month_Jul,Month_June,Month_Mar,Month_May,Month_Nov,Month_Oct,Month_Sep,VisitorType_Other,VisitorType_Returning_Visitor,Revenue
0,0.000000,0.0,0.0,0.0,0.001418,0.000000,1.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,0.000000,0.0,0.0,0.0,0.002837,0.001000,0.000000,0.500000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,0.000000,0.0,0.0,0.0,0.001418,0.000000,1.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,0.000000,0.0,0.0,0.0,0.002837,0.000042,0.250000,0.700000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,0.000000,0.0,0.0,0.0,0.014184,0.009809,0.100000,0.250000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
5,0.000000,0.0,0.0,0.0,0.026950,0.002411,0.078947,0.122807,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
6,0.000000,0.0,0.0,0.0,0.001418,0.000000,1.000000,1.000000,0.0,0.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
7,0.037037,0.0,0.0,0.0,0.000000,0.000000,1.000000,1.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
8,0.000000,0.0,0.0,0.0,0.002837,0.000578,0.000000,0.500000,0.0,0.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
9,0.000000,0.0,0.0,0.0,0.004255,0.011536,0.000000,0.111111,0.0,0.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [40]:
# spaces.box??
# spaces.Discrete??

In [41]:
Reward_adjustment = 1
# Refers to Reward structure. Is imbalance Ratio of classes.
# This ratio/reward maybe tuned to improve model Recall
lamb_da = (1908/10422) * Reward_adjustment

In [42]:
lamb_da

0.18307426597582038

In [43]:
class DataFrameEnv(gym.Env):
 
    def __init__(self):
        
        #Init Action Space with to Discrete steps. 0 or 1, Left or right, On or off.
        self.action_space = spaces.Discrete(2)
        #Init Observation space 26x1 that contain continous values. change shape to fit any dataframe(in theory)
        self.observation_space = spaces.Box(-np.inf, np.inf, shape=(26,), dtype=np.float32)
        
        #Init the rest of DataFrameEnv class variables
        self.episode_df = None
        self.viewer = None
        self.state = None

        self.steps_beyond_done = None
        self.episode = None
        self.test_set_no_labels = None
        self.test_labels = None

    def reset(self):
        # Reset function randomly samples Dataframe, Splits off labels and converts to list
        self.episode_df = df.sample(frac=0.8)
        self.episode = self.episode_df.drop(columns=['Revenue']).values.tolist()
        self.true_labels = self.episode_df['Revenue'].values.tolist()
        
        try:
            # Remove one row from sampled DF 
            self.state = self.episode.pop()
        except:
            self.state = None

        self.steps_beyond_done = None
        # Return to agent as a state.
        return np.array(self.state)

    
    def step(self, action):
        assert self.action_space.contains(action), "%r (%s) invalid"%(action, type(action))
        
        # Pull off next row and assign to state to be returned to agent from this function
        state = self.episode.pop()
        

        done = False
        true_label = self.true_labels.pop()
        # Compare agents prediction with label and assign reward
        
        if true_label is not None:
            
            if (true_label == 1) & (action == true_label):
                reward = 1
            
            elif (true_label == 0) & (action == true_label):
                reward = lamb_da
            
            elif (true_label == 0) & (action != true_label):
                reward = -lamb_da
            
            else:
                reward = -1
                done = True
        else:
            reward = 0
            done = True
        # Return Next state, reward for action in current state and if done/ stop episode.
        return np.array(state), reward, done, {}

    def validate(self):
        # Validation function compare predictions to actuals on test set to build Confusion Matrix
        test_set = df[~df.index.isin(self.episode_df.index)]
        self.test_set_no_labels = test_set.drop(columns=['Revenue'])
        self.test_labels = test_set['Revenue']
        
        return self.test_set_no_labels, self.test_labels

In [44]:
# Init Environment
env = DataFrameEnv()

In [45]:
# Hyperparameters
H = 52 # number of hidden layer neurons
batch_size = 6 # how many episodes before doing a policy update
learning_rate = 0.009 # How big/small the steps are in the gradient ascent
gamma = 0.99 # discount factor for reward
learning_decay = 50 #Learning rate decay (not used currently)
D = 26 # input dimensionality
total_episodes = 1000 #duhh

In [46]:
def discount_rewards(r):
    # Calculates a reward to reduce rewards gained towards the end of an episode.
    # Not used except for gamma which is used to regulate prioritization of current vs future rewards.
    discounted_r = np.zeros_like(r)
    running_add = 0
    for t in reversed(xrange(0, r.size)):
        running_add = running_add * gamma + r[t]
        discounted_r[t] = running_add
    return discounted_r

In [47]:
tf.compat.v1.reset_default_graph()

#This defines the network as it goes from taking an observation of the environment to 
#giving a probability of chosing to the action 0 or 1.
observations = tf.compat.v1.placeholder(tf.float32, [None,D] , name="input_x")
W1 = tf.compat.v1.get_variable("W1", shape=[D, H],
           initializer=tf.initializers.GlorotUniform())
# first layer activation function was Tanh for PCA inputs to capture negative value information 
# changed to relu for modelling on original scaled dataset
layer1 = tf.nn.relu(tf.matmul(observations,W1))
W11 = tf.compat.v1.get_variable("W11", shape=[H, 8],
           initializer=tf.initializers.GlorotUniform())


layer11 = tf.nn.relu(tf.matmul(layer1, W11))
W2 = tf.compat.v1.get_variable("W2", shape=[8, 1],
           initializer=tf.initializers.GlorotUniform())

score = tf.matmul(layer11,W2)

probability = tf.nn.sigmoid(score)

#From here we define the parts of the network needed for learning a good policy.
tvars = tf.compat.v1.trainable_variables()
input_y = tf.compat.v1.placeholder(tf.float32,[None,1], name="input_y")
advantages = tf.compat.v1.placeholder(tf.float32,name="reward_signal")

# The loss function. This sends the weights in the direction of making actions 
# that gave good advantage (reward over time) more likely, and actions that didn't less likely.
loglik = tf.compat.v1.log(input_y*(input_y - probability) + (1 - input_y)*(input_y + probability))
loss = -tf.reduce_mean(loglik * advantages)

newGrads = tf.gradients(loss,tvars)

# Once we have collected a series of gradients from multiple episodes, we apply them.
# We don't just apply gradeients after every episode in order to account for noise in the reward signal.
adam = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate) # Our optimizer
W1Grad = tf.compat.v1.placeholder(tf.float32,name="batch_grad1")
W11Grad = tf.compat.v1.placeholder(tf.float32,name="batch_grad11")# Placeholders to send the final gradients through when we update.
W2Grad = tf.compat.v1.placeholder(tf.float32,name="batch_grad2")
batchGrad = [W1Grad,W11Grad,W2Grad]
updateGrads = adam.apply_gradients(zip(batchGrad,tvars))

In [48]:
#stores states, actions, rewards for an episode
xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[]
running_reward = None
reward_sum = 0
episode_number = 1
episode_rewards = []

#lists that maybe used to visualise model training/updates
policy_gradient = []
Q_approx = []
loss_list = []
predictions = []

In [49]:
init = tf.compat.v1.global_variables_initializer()
# Launch the graph
with tf.compat.v1.Session() as sess:
    
    sess.run(init)
    observation = env.reset() # Obtain an initial observation of the environment

    # Reset the gradient placeholder. We will collect gradients in 
    # gradBuffer until we are ready to update our policy network. 
    gradBuffer = sess.run(tvars)
    for ix,grad in enumerate(gradBuffer):
        gradBuffer[ix] = grad * 0
    
    while episode_number <= total_episodes:
        
        # Make sure the observation is in a shape the network can handle.
        x = np.reshape(observation,[1,D])
        
        # Run the policy network and get an action to take. 
        tfprob = sess.run(probability,feed_dict={observations: x})
        action = 1 if np.random.uniform() < tfprob else 0
        
        
        xs.append(x) # observation
        y = 1 if action == 0 else 0 # a "fake label"
        ys.append(y)

        # step the environment and get new measurements
        observation, reward, done, info = env.step(action)
        
        reward_sum += reward

        drs.append(reward) # record reward (has to be done after we call step() to get reward for previous action)
        
        
        if done: 
            episode_number += 1
            # stack together all inputs, hidden states, action gradients, and rewards for this episode
            epx = np.vstack(xs)
            epy = np.vstack(ys)
            epr = np.vstack(drs)
            episode_rewards.append(np.sum(drs))
            tfp = tfps
            xs,hs,dlogps,drs,ys,tfps = [],[],[],[],[],[] # reset array memory

            # compute the discounted reward backwards through time
            discounted_epr = discount_rewards(epr)
            
            
            # Get the gradient for this episode, and save it in the gradBuffer
            tGrad = sess.run(newGrads,feed_dict={observations: epx, input_y: epy, advantages: discounted_epr})
            for ix,grad in enumerate(tGrad):
                gradBuffer[ix] += grad
                
            # If we have completed enough episodes, then update the policy network with our gradients.
            if episode_number % batch_size == 0: 
                policy_gradient.append(gradBuffer)
                sess.run(updateGrads,feed_dict={W1Grad: gradBuffer[0],W11Grad:gradBuffer[1],W2Grad:gradBuffer[2]})

                loss_list.append(loss)
                for ix,grad in enumerate(gradBuffer):
                    gradBuffer[ix] = grad * 0
                
                # Give a summary of how well our network is doing for each batch of episodes.
                running_reward = reward_sum if running_reward is None else running_reward * 0.99 + reward_sum * 0.01
                
                print('Average reward for episode %f.  Total average reward %f.' % (reward_sum//batch_size, running_reward//batch_size))
                
                reward_sum = 0

            observation = env.reset()
    #get test set with labels. 
    test_set, test_labels = env.validate()
    
    for i in test_set.values:
        # predict on each test row and append to list
        prediction = sess.run(probability, feed_dict={observations:np.reshape(i,[1,D])})
        predictions.append(prediction)
        
            
print(episode_number,'Episodes completed.')


Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode 0.000000.  Total average reward -1.000000.
Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode 0.000000.  Total average reward -1.000000.
Average reward for episode 0.000000.  Total average reward -1.000000.
Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode -1.000000.  Total average reward -1.000000.
Average reward for episode 0.000000.  Total average reward -1.000000.
Average reward for episode 0.000000.  Total average reward -1.000000.
Average rew

In [50]:
episode_number

1001

In [51]:
pred_clean = []
for i in predictions:
    if i < .50:
        i = 0
        pred_clean.append(i)
    else:
        i = 1
        pred_clean.append(i)

In [54]:
cf_report_1 = classification_report(test_labels,pred_clean)

In [55]:
print(cf_report_1)

              precision    recall  f1-score   support

           0       1.00      0.19      0.31      2054
           1       0.20      1.00      0.33       412

    accuracy                           0.32      2466
   macro avg       0.60      0.59      0.32      2466
weighted avg       0.86      0.32      0.32      2466



In [19]:
#test_labels

In [20]:
#pred_clean

In [53]:
from sklearn.metrics import confusion_matrix, classification_report

In [22]:
cf_35_reward = confusion_matrix(test_labels,pred_clean,normalize='all')

In [23]:
cf_35_reward

array([[0.4703974 , 0.37793998],
       [0.0081103 , 0.14355231]])

In [24]:
cf_3_reward

NameError: name 'cf_3_reward' is not defined

In [ ]:
cf_2_reward

In [29]:
cf_report_35 = classification_report(test_labels,pred_clean)

In [30]:
print(cf_report_35)

              precision    recall  f1-score   support

           0       0.98      0.55      0.71      2092
           1       0.28      0.95      0.43       374

    accuracy                           0.61      2466
   macro avg       0.63      0.75      0.57      2466
weighted avg       0.88      0.61      0.67      2466



In [59]:
from lightgbm import LGBMClassifier
from torch.nn import RNN
from sklearn.model_selection import train_test_split

In [57]:
RNN??

Init signature: RNN(*args, **kwargs)
Source:        
class RNN(RNNBase):
    r"""Applies a multi-layer Elman RNN with :math:`tanh` or :math:`ReLU` non-linearity to an
    input sequence.


    For each element in the input sequence, each layer computes the following
    function:

    .. math::
        h_t = \text{tanh}(W_{ih} x_t + b_{ih} + W_{hh} h_{(t-1)} + b_{hh})

    where :math:`h_t` is the hidden state at time `t`, :math:`x_t` is
    the input at time `t`, and :math:`h_{(t-1)}` is the hidden state of the
    previous layer at time `t-1` or the initial hidden state at time `0`.
    If :attr:`nonlinearity` is ``'relu'``, then `ReLU` is used instead of `tanh`.

    Args:
        input_size: The number of expected features in the input `x`
        hidden_size: The number of features in the hidden state `h`
        num_layers: Number of recurrent layers. E.g., setting ``num_layers=2``
            would mean stacking two RNNs together to form a `stacked RNN`,
            with the sec

In [58]:
lgb_classifer = LGBMClassifier()

In [60]:
df.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,...,Month_Jul,Month_June,Month_Mar,Month_May,Month_Nov,Month_Oct,Month_Sep,VisitorType_Other,VisitorType_Returning_Visitor,Revenue
0,0.0,0.0,0.0,0.0,0.001418,0.000000,1.00,1.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
1,0.0,0.0,0.0,0.0,0.002837,0.001000,0.00,0.50,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,0.0,0.0,0.0,0.0,0.001418,0.000000,1.00,1.00,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
3,0.0,0.0,0.0,0.0,0.002837,0.000042,0.25,0.70,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,0.0,0.0,0.0,0.0,0.014184,0.009809,0.10,0.25,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0


In [64]:
X = df.drop('Revenue',axis=1)
y = df.Revenue

In [65]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [66]:
X_train

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,...,Month_Feb,Month_Jul,Month_June,Month_Mar,Month_May,Month_Nov,Month_Oct,Month_Sep,VisitorType_Other,VisitorType_Returning_Visitor
4079,0.000000,0.000000,0.000000,0.0,0.004255,0.000141,0.333333,0.666667,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
11898,0.074074,0.062817,0.000000,0.0,0.008511,0.001703,0.000000,0.125000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2455,0.000000,0.000000,0.000000,0.0,0.029787,0.009131,0.000000,0.035714,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
11581,0.000000,0.000000,0.000000,0.0,0.001418,0.000000,1.000000,1.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
7647,0.370370,0.241582,0.041667,0.0,0.078014,0.037528,0.000000,0.053586,0.002712,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1061,0.000000,0.000000,0.000000,0.0,0.004255,0.000188,0.000000,0.333333,0.000000,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
10749,0.518519,0.058968,0.000000,0.0,0.009929,0.001132,0.000000,0.023810,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3419,0.000000,0.000000,0.000000,0.0,0.043972,0.016991,0.000000,0.033333,0.000000,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1288,0.037037,0.000000,0.000000,0.0,0.024113,0.020544,0.229167,0.500000,0.000000,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0


In [68]:
lgb_classifer.fit(X_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [69]:
lgb_classifer.score(X_test,y_test)

0.9033409017191047

In [73]:
y_pred = lgb_classifer.predict(X_test)

In [75]:
lgb_clas_report = classification_report(y_test,y_pred)

In [76]:
print(lgb_clas_report)

              precision    recall  f1-score   support

           0       0.93      0.95      0.94      2628
           1       0.70      0.61      0.65       455

    accuracy                           0.90      3083
   macro avg       0.82      0.78      0.80      3083
weighted avg       0.90      0.90      0.90      3083



In [77]:
from sklearn.linear_model import LogisticRegression

In [78]:
log_reg = LogisticRegression()

In [79]:
log_reg.fit(X_train,y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [80]:
log_reg.score(X_test,y_test)

0.8871229322088875

In [81]:
log_y_pred = log_reg.predict(X_test)

In [85]:
log_reg_clas_report = classification_report(log_y_pred,y_test)

In [86]:
print(log_reg_clas_report)

              precision    recall  f1-score   support

           0       0.99      0.89      0.94      2916
           1       0.30      0.82      0.44       167

    accuracy                           0.89      3083
   macro avg       0.64      0.86      0.69      3083
weighted avg       0.95      0.89      0.91      3083

